### 1. 필요한 라이브러리들을 불러옵니다. 

In [17]:
from selenium import webdriver
from bs4 import BeautifulSoup
import sqlite3
import time

### 2. 셀레니움 웹드라이버를 이용해서 다음 카페 모바일의 로그인 페이지를 열어줍니다. 

In [18]:
driver = webdriver.Chrome('C:/PyApp/driver/chromedriver')
# 설치 폴더에 주의합니다. 

driver.get('https://logins.daum.net/accounts/loginform.do?mobilefull=1&category=cafe&url=http%3A%2F%2Fm.cafe.daum.net%2F_myCafe%3Fnull')

time.sleep(5)
# 페이지 전환시에는 적당한 시간을 줍니다. 
# 1. 과도한 크롤링 방지.
# 2. 페이지 전환이 완료되기 전에 다음 명령 실행되는 것 방지.

### 3. 아이디와 비밀 번호 입력

In [19]:
# 수동으로 웹 브라우저에 직접 입력 후 건너뛰어도 됩니다. 

driver.find_element_by_xpath("""//*[@id="id"]""").send_keys('midikid') # id를 입력한다. 
driver.find_element_by_xpath("""//*[@id="inputPwd"]""").send_keys('tkfka95@dm') # 패스워드를 입력한다. 
driver.find_element_by_xpath("""//*[@id="loginBtn"]""").click() # 입력 버튼 클릭.

time.sleep(5)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=74.0.3729.131)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Windows NT 10.0.17763 x86_64)


### 4. 까페 이름, 게시판 주소 등을 보기 좋게 여기쯤 지정해 줍니다. 

In [ ]:
CAFE_NAME = 'ssaumjil' 
# 까페 이름을 넣어준다. 예제는 이종격투기... 

BOARD_NAME = 'EnXG' 
# 게시판 주소의 마지막 4자리를 넣어준다. 
# http://m.cafe.daum.net/ssaumjil/EnXG

DB = CAFE_NAME + '_m_article.db'
# DB 이름도 지정해 줍니다. 

### 5. 게시판으로 이동

### 6. 데이터베이스 초기화

In [12]:
conn = sqlite3.connect(DB)
cur = conn.cursor()

sql = 'CREATE TABLE IF NOT EXISTS "' + BOARD_NAME + \
  '"("num" INTEGER NOT NULL UNIQUE, "subject" TEXT, ' + \
  '"nick" TEXT, "write_time" TEXT, "views" INTEGER, ' + \
  '"url" TEXT, "contents" TEXT, PRIMARY KEY("num"));'

cur.execute(sql)
conn.commit()

sql = 'CREATE TABLE IF NOT EXISTS "' + BOARD_NAME + \
  '_cmt" ("cmt_num" TEXT NOT NULL UNIQUE, "cmt_reply" INTEGER, "cmt_writer" TEXT, "cmt_time" TEXT, "cmt_txt" TEXT, PRIMARY KEY("cmt_num"));'

cur.execute(sql)
conn.commit()

conn.close()

### 7. 캡쳐

In [15]:
inp_num = input('저장할 게시물 번호 + 엔터: ')

num = int(inp_num) #뒤에 DB에 정수로 저장할 거니까 형변환 해줍시다. 

url = 'http://m.cafe.daum.net/%s/%s/%s' % (CAFE_NAME, BOARD_NAME, inp_num)
driver.get(url) # 게시물의 주소로 이동합니다. 

time.sleep(5)

#### 이제 다음 줄에서 페이지 소스를 얻은 것을 마지막(?)으로 셀레니움의 역할은 끝납니다. 

In [ ]:
html = driver.page_source  # 셀레니움의 페이지 소스를 html로... 
soup = BeautifulSoup(html, 'html.parser') 
# 뷰티플 숩은 html을 받아서 html 파서를 돌려서 soup로 보냅니다. 

# repr(soup)

#### 게시물의 구성요소들을 하나씩 찾아봅니다. 

subject = soup.body.find('h3', class_='tit_subject')

# 숩은 엘리멘트 트리와 달리 class명을 이용해서 h3을 바로 찾아줍니다. 
# 같은 태그에 클래스명까지 겹치면 겹치는 노드 중 첫번째 노드가 검색됩니다. 
# 이 HTML 문서에서 'tit_subject' 클래스는 한번만 사용됩니다. ^^
# 개발자 도구의 검색 기능을 이용해 보십시오. 

if subject is None:
    # None 객체에 .get_text() 메소드를 붙이면 에러가 발생합니다. 
    # 연속 크롤링시에는 에러를 최대한 피해줘야 합니다. 
    # 그래서 if 문으로 사전에 걸러주는 겁니다. 
    # 대신 코드가 지저분해지기 시작합니다. 
    print(url, '지워진 게시물입니다.')
else:
    # 지워진 게시물이 아닌 경우 이하를 계속 실행합니다. 
    subject = subject.get_text(strip=True)  
    
    # 뷰티플 숩은 XPath를 지원하지 않습니다. 
    # 작성자 항목은 직접적인 태그로 감싸지지도 않았습니다. 
    # 이런 경우에는 옆 노드를 찾아서 그 노드의 이웃으로 찾아야 합니다. 
    # XPath의 편리함이 떠오를 따름입니다. ㅠ,.ㅠ 

    # 다음은 익명 게시판기능도 있어서 작성자가 공란인 경우도 있습니다. 
    # 미리 걸러줘야 합니다. 
    
    if soup.body.find('span', class_='txt_subject').find('span', class_='sr_only').get_text() == '작성자':
        nick = soup.body.find('span', class_='sr_only').next_sibling
    else:
        nick = ''

    # 작성시간과 조회수를 찾아봅시다. 
    # 클래스 명이 지정되어 있습니만, 
    # 작성시간, 조회수 2군데에서 사용되었네요. 
    # 이런 경우는 find_all()로 작성시간과 조회수를 동시에 찾아낸 뒤 
    # [0], [1]로 리스트의 요소를 하나씩 뽑아내야 겠네요. 

    # num_subject라는 변수명은 해당 노드의 클래스명에서 따왔습니다. 
    # 프로그래머의 가장 큰 고민은 변수 이름을 짓는 것이라고 합니다.  
    # 원본을 그대로 쓰면 그런 고민을 할 필요가 없어집니다. 
    # 하지만 .... 직관적이지 않은 변수명 같습니다...
        
    num_subject = soup.body.find_all('span', class_='num_subject')
    write_time = num_subject[0].get_text()
    views = num_subject[1].get_text()

    print(num, subject, nick, write_time, views, url)

    # 본문
    
    # 본문은 친절하게 id='article' 안에 잘 들어있습니다. 
    # 이런 경우는 id는 유일하니까 고민이 별로 없어집니다. 
    # 바로 find로 찾으면 됩니다. 

    contents = soup.body.find('div', id='article').get_text('\n', strip=True)
    print(contents)

    
    # 위에서 사용했던 변수들을 모아서 DB에 넣어줍니다.
    # DB에 관한 설명은 별도로 하지 않겠습니다.  
    # 제가 첫 시간에 알려드렸던 사이트들을 보시면 될 것 같습니다. 
    
    conn = sqlite3.connect(DB)
    cur = conn.cursor()
    sql = "replace into %s(num,subject,nick,write_time,views,url,contents) values (?,?,?,?,?,?,?)" % BOARD_NAME
    cur.execute(sql, (num, subject, nick, write_time, views, url, contents))
    conn.commit()
    conn.close()
    
    # 댓글 개수
    cmt_total = soup.body.find('article', id='mArticle').find('span', class_="num_total").get_text()
    print('댓글 개수 :', cmt_total)

    # 댓글

    # 댓글 페이지로 이동
    driver.get('http://m.cafe.daum.net/{0}/{1}/{2}/comments'.format(CAFE_NAME,BOARD_NAME,inp_num))
    time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 크롤링한 페이지에서 페이지 슷자 추출(cmt_index)
    cmt_page_max = int(soup.body.find('span', id="pagingNav").find_all('span', class_="num_page").pop().get_text())

    print('cmt_page_max', cmt_page_max)
    for i in range(cmt_page_max):
        j = cmt_page_max-i
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        cmt_all = soup.body.find('ul' ,id="commentList").find_all('li')
        
        for k in cmt_all:
            if k.div.span.find_all('span', class_="txt_bar"):
                if k['class'] == ['reply_on']:
                    cmt_reply = True
                    print('ㄴ', end='')
                else:
                    cmt_reply = False

                cmt_num = inp_num +'_'+ k['id'].split('comment_')[1]
                cmt_writer = k.div.find('span', class_='sr_only').next_sibling
                cmt_time = k.div.span.find('span', class_="num_info").get_text()
                cmt_txt = k.div.find('span', class_='txt_detail').get_text(strip=True)
                print(cmt_num, '[', cmt_writer, ']', cmt_time, cmt_txt)
                
                conn = sqlite3.connect(DB)
                cur = conn.cursor()
                sql = "replace into '%s_cmt'(cmt_num,cmt_reply,cmt_writer,cmt_time,cmt_txt) values (?,?,?,?,?)" % BOARD_NAME
                cur.execute(sql, (cmt_num,cmt_reply,cmt_writer,cmt_time,cmt_txt))
                conn.commit()
                conn.close()
                
        # print(j)
        if j > 1:
            driver.get('http://m.cafe.daum.net/{0}/{1}/{2}/comments?prev_page={3}&mode=regular&cdepth={4}&page={5}'.format(CAFE_NAME,BOARD_NAME,inp_num,j,'0002100000',(j-1)))
            time.sleep(2)

In [101]:
driver.close()
# 열었던 브라우저를 닫아줍니다. 